# 1. Starting and Stopping Threads

## Problem: Create and destroy threads for concurrent execution of code.

In [4]:
# Creating simple thread

import time

def countdown(n):
    while n > 0:
        print('Count ', n)
        n -= 1
        time.sleep(5)
        
from threading import Thread
t = Thread(target=countdown, args = (10, ))
t.start()

Count  10


In [12]:
# Simple thread with termination functionality

class CountdownTask():
    def __init__(self):
        self._running = True
        
    def terminate(self):
        self._running = False
    
    def run(self, n):
        while self._running and n > 0:
            print('Count ', n)
            n -= 1
            time.sleep(3)

c = CountdownTask()
t = Thread(target=c.run, args=(5, ))
t.start()
time.sleep(6)
c.terminate()
t.join()

Count  5
Count  4


In [14]:
# Creating process

import multiprocessing
c = CountdownTask()
p = multiprocessing.Process(target=c.run, args=(4, ))
p.start()

Count  4
Count  3
Count  2
Count  1


# 2. Determining If a Thread Has Started

## Problem: You’ve launched a thread, but want to know when it actually starts running.

In [19]:
# Using Event

from threading import Thread, Event
import time

def countdown(n, started_evt):
    print('Countdown thread started')
    started_evt.set()
    while n > 0:
        print('Count ', n)
        n -= 1
        time.sleep(3)

started_evt = Event()

print('Starting countdown')
t = Thread(target=countdown, args=(9, started_evt))
t.start()
started_evt.wait()
print('Main thread started')

Starting countdown
Countdown thread started
Main thread startedCount 
 9
Count  1
Count  8
Count  7
Count  6
Count  5
Count  4
Count  3
Count  2
Count  1


In [24]:
# Using Condition object

import threading
import time

class PeriodicTimer:
    def __init__(self, interval):
        self._interval = interval
        self._flag = 0
        self._cv = threading.Condition()
        
    def start(self):
        t = threading.Thread(target=self.run)
        t.daemon = True
        t.start()
        
    def run(self):
        while True:
            time.sleep(self._interval)
            with self._cv:
                self._flag ^= 1
                self._cv.notify_all()
                
    def wait_for_tick(self):
        with self._cv:
            last_flag = self._flag
            while last_flag == self._flag:
                self._cv.wait()
                
p_timer = PeriodicTimer(5)
p_timer.start()

def countdown(c):
    while c > 0:
        p_timer.wait_for_tick()
        print('Count Down ', c)
        c -= 1
        
def countup(c):
    n = 1
    while n <= c:
        p_timer.wait_for_tick()
        print('Count Up ', n)
        n += 1

threading.Thread(target=countdown, args=(10,)).start()
threading.Thread(target=countup, args=(10,)).start()
                

Count Down Count Up   101

Count Down Count Up   92

Count Down Count Up   83

Count Down Count Up   47

Count Down Count Up  6 
5
Count Down Count Up   65

Count Down Count Up   74

Count Down Count Up   38

Count Up Count Down   92

Count Up Count Down   101



In [25]:
# Using semaphore

def worker(n, sema):
    sema.acquire()
    print('Working ', n)
    
sema = threading.Semaphore(0)
nworkers = 10

for n in range(nworkers):
    t = threading.Thread(target=worker, args=(n, sema,))
    t.start()

# Communicating between Threads